####  1. 数据处理

可以看到数据是典型的BIO文件，文本类型为txt

In [ ]:
import pandas as pd

def load_bio_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    sentences = []
    current_sentence = ""
    
    label={}
    
    temp_label_name = ""
    for line in lines:
        # print(line)
        line = line.replace('\n','')
        line = line.strip()
        # print(line)
        if not line:  # 空行表示句子结束
            if current_sentence:
                # 处理最后一个词
                if len(temp_label_name)>0:
                    label[temp_label_name] = temp_label_value
                    temp_label_name=""
                sentences.append({'text':current_sentence,'label':label})
                current_sentence = ""
                label={}
            continue
        
        parts = line.split()
        current_sentence = current_sentence + parts[0]  # 字符
        if len(parts)>=2 and parts[1] == 'O' and len(temp_label_name)>0:
            label[temp_label_name] = temp_label_value
            temp_label_name=""
        elif len(parts)>=2 and parts[1][0] =='B':
            temp_label_name = parts[0]
            temp_label_value =parts[1][2:]
        elif len(parts)>=2 and parts[1][0] =='I':
            temp_label_name = temp_label_name + parts[0]
            
    # 处理最后一个句子
    if (current_sentence, label):
        sentences.append({'text':current_sentence,'label':label})
    
    return sentences

# 示例加载
train = load_bio_data('data/medical.train')
dev = load_bio_data('data/medical.dev') # 跑不动我这回就不用了
test = load_bio_data('data/medical.dev') # 跑不动我这回就不用了

In [92]:
str(train[2])

"{'text': '舒肝和胃消痞汤；功能性消化不良', 'label': {'功能性消化不良': '西医诊断'}}"

In [50]:
from datasets import Dataset
train_ds = Dataset.from_list(train)

In [ ]:
# 缩减数据长度 在tokenizer中配置max_length 以减轻训练负担
train_150 = []
max_len= 0 
for i in train:
    len_items = len(i['text']) +len(str(i['label'])) 
    max_len = max(max_len,len_items)
    if len_items<=150:
        train_150.append(i)
print("原始样本量:",len(train),"缩减后:",len(train_150))
print("原始样本最大长度:",max_len,"缩减后样本长度:",150)

原始样本量: 5259 缩减后: 4879
原始样本最大长度: 358 缩减后样本长度: 150


In [73]:
# 缩减数据长度 在tokenizer中配置max_length 以减轻训练负担
labels_name = set()
for i in train:
    for k in i['label']:
        labels_name.add(i['label'][k])
print(labels_name)

{'中医诊断', '中药', '中医治疗', '方剂', '西医治疗', '西医诊断', '中医治则', '中医证候', '临床表现', '其他治疗'}


In [91]:
from modelscope import snapshot_download, AutoTokenizer
from transformers import AutoModelForCausalLM
import torch
# 1. 加载模型
model_id = "qwen/Qwen2-1.5B-Instruct"    
model_dir = "/Users/luyi/PythonProjects/Atomy/03LLM微调/命名实体微调实战/qwen/Qwen2-1___5B-Instruct"

# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", torch_dtype=torch.bfloat16)
# model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

#### 未训练前测试一下效果

In [95]:
def predict(messages, model, tokenizer):
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to('mps')

    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=model_inputs.attention_mask, # 加这一行避免报错：Please provide an `attention_mask` or use a different device.
        max_new_tokens=300
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    print(response)
     
    return response

In [96]:
def testModel(model,instruction,input_value,tokenizer):
    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}
    ]
    return predict(messages, model, tokenizer)
instruction = """你是一个文本实体识别领域的医学专家，你需要从给定的句子中提取中医诊断,中药,中医治疗, 方剂, 西医治疗, 西医诊断 '其他治疗'等. 以 json 格式输出, 如 {'口苦': '临床表现','肺结核': '西医诊断'} 注意: 1. 输出的每一行都必须是正确的json字符串. 2.找不到任何实体时, 输出"没有找到任何实体"."""
input_value = '舒肝和胃消痞汤；功能性消化不良'
testModel(model,instruction,input_value,tokenizer)

['舒肝和胃消痞汤']


"['舒肝和胃消痞汤']"

In [97]:
# 返回一个字典，包括 input_ids, attention_mask, labels
# labels中的值是 input_ids 中 attention_mask为1 然后 -100处理后 的值
def process_func(example):
    MAX_LENGTH = 310 # 166 + 150 = 316 系统提示长度 + 最长的样本长度
    system_prompt = """你是一个文本实体识别领域的医学专家，你需要从给定的句子中提取中医诊断,中药,中医治疗, 方剂, 西医治疗, 西医诊断 '其他治疗'等. 以 json 格式输出, 如 {'口苦': '临床表现','肺结核': '西医诊断'} 注意: 1. 输出的每一行都必须是正确的json字符串. 2.找不到任何实体时, 输出"没有找到任何实体"."""
    input_ids, attention_mask, labels = [], [], [] # 最终返回的字典就这三个值
    
    instruction = tokenizer(
        f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{example['text']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['label']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = (
        instruction["attention_mask"] + response["attention_mask"] + [1]
    )
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}  

In [ ]:
from datasets import Dataset
train_ds = Dataset.from_list(train_150)
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)

Map:   0%|          | 0/4879 [00:00<?, ? examples/s]

0.16s - Expected: /opt/anaconda3/envs/tunning/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd_attach_to_process/attach.dylib to exist.


In [ ]:
# 导包
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Lora微调参数
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    # target_modules=["q_proj",  "v_proj", "o_proj", "down_proj"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

model_lora = get_peft_model(model, config)

# 训练参数
args = TrainingArguments(
    output_dir="./output/Qwen2-NER-Doctor",
    per_device_train_batch_size=4, # 每张显卡上 的 batch_size 一个GPU 就是4
    gradient_accumulation_steps=8, # 累计多少步更新一次参数，通常会比per_device_train_batch_size大
    logging_steps=10,
    num_train_epochs=2,
    save_steps=50,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
    # bf16=True,          # 👈 关键！M1 必开
    # fp16=False,         # MPS 不支持 fp16，必须用 bf16
)

In [ ]:
#设置 Trainer 开始训练
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

trainer.train()